High Performance Python
=======

Lecture 8. Numba
----------

Heavily based on (forked):

Scipy2017 tutorial by Gil Forsyth:

https://github.com/gforsyth/numba_tutorial_scipy2017

https://www.youtube.com/watch?v=1AwG0T4gaO0&t=1349s

GTC2018 tutorial by Stan Seibert:

https://github.com/ContinuumIO/gtc2018-numba


High Performance Python
----------------------

* multiprocessing
* mpi4py
* pycuda
* cupy
* pyopencl
* **numba**

In [ ]:
2341287430918682137649812736498176149823746918273469182736498172364981762394871263948716239847612983476192834671982374619827346918273641982736419827367491827364182736498127364981726349817263498716239847612398746 * 102983471023874109238741092837401928374091283740129837409128374012837409129387409182734091837

241113906300574757165921605499194544451748291802645407881299520206724413881776447095788703691261538743613810971029915964006947544477386524396492216174382355640855545991459366716679056706024737473592418809044329692985397166084782002332540365120191225689906538421177259996756621053527936712071631427636402

Numba is:

Just-In-Time (JIT) compiler:
* generates optimized machine code using LLVM
* integrates well with Scientific Python stack
* **function compiler**: Numba compiles Python functions (not entire applications and not parts of functions). Numba is a Python module.
* **type-specializing**: Numba speeds up your function by generating a specialized implementation for the specific data types you are using.
* **just-in-time**: Numba translates functions when they are first called so that the compiler knows the argument types. Works in Jupyter notebook.
* **numerically-focused**: „mostly“ int, float, complex. Works good with numpy arrays.


AOT

The first step is always to find the bottlenecks in your code, via _profiling_: analyzing your code by measuring the execution time of its parts.


Tools:
------

2. `cProfile`
4. `snakeviz`
1. [`line_profiler`](https://github.com/rkern/line_profiler)
3. `timeit`



```console
pip install line_profiler
```

In [ ]:
import numpy
from time import sleep

def sleepy(time2sleep):
    sleep(time2sleep)

def supersleepy(time2sleep):
    sleep(time2sleep)

def randmatmul(n=1000):
    a = numpy.random.random((n,n))
    b = a @ a
    return b

def useless(a):
    if not isinstance(a, int):
        return

    randmatmul(a)

    ans = 0
    for i in range(a):
        ans += i

    sleepy(1.0)
    supersleepy(2.0)

    return ans

## using `cProfile`

[`cProfile`](https://docs.python.org/3.4/library/profile.html#module-cProfile) is the built-in profiler in Python (available since Python 2.5).  It provides a function-by-function report of execution time. First import the module, then usage is simply a call to `cProfile.run()` with your code as argument. It will print out a list of all the functions that were called, with the number of calls and the time spent in each.


In [ ]:
import cProfile

cProfile.run('useless(3000)')

## using `snakeviz`

In [ ]:
%load_ext snakeviz

In [ ]:
%snakeviz useless(3000)

## using `line_profiler`

`line_profiler` offers more granular information than `cProfile`: it will give timing information about each line of code in a profiled function.

### For a pop-up window with results in notebook:

IPython has an `%lprun` magic to profile specific functions within an executed statement. Usage:
`%lprun -f func_to_profile <statement>` (get more help by running `%lprun?` in IPython).

In [ ]:
%load_ext line_profiler
%lprun -f sleepy -f supersleepy useless(1000)

### Write results to a text file

In [ ]:
%lprun -T timings.txt -f sleepy useless(1000)

## Profiling on the command line

Open file, add `@profile` decorator to any function you want to profile, then run

```console
kernprof -l script_to_profile.py
```

which will generate `script_to_profile.py.lprof` (pickled result).  To view the results, run

```console
python -m line_profiler script_to_profile.py.lprof
```

In [ ]:
from IPython.display import IFrame
IFrame('http://localhost:8888/terminals/1', width=800, height=700)

## `timeit`

```python
python -m timeit "print(42)"
```


In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

In [ ]:
# line magic
%timeit x=10

20.5 ns ± 0.154 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [ ]:
%%timeit
# cell magic

x=10
a='hello'
d=[1,2,3]

76.4 ns ± 0.997 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


JIT
===

### Array sum

The function below is a naive `sum` function that sums all the elements of a given array.

In [ ]:
def sum_array(inp):
    J, I = inp.shape

    #this is a bad idea
    mysum = 0
    for j in range(J):
        for i in range(I):
            mysum += inp[j, i]

    return mysum

In [ ]:
import numpy
arr = numpy.random.random((300, 300))

sum_array(arr)

plain = %timeit -o sum_array(arr)

20.4 ms ± 615 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
from numba import jit

sum_array_numba = jit()(sum_array)

sum_array_numba(arr)

jitted = %timeit -o sum_array_numba(arr)

plain.best / jitted.best


141 µs ± 2.06 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


141.59957032021785

## More commonly as a decorator

In [ ]:
@jit
def sum_array(inp):
    I, J = inp.shape

    mysum = 0
    for i in range(I):
        for j in range(J):
            mysum += inp[i, j]

    return mysum

In [ ]:
%timeit arr.sum()

44.7 µs ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
%timeit numpy.sum(arr)

52.7 µs ± 12.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Exercise 1. JIT the Mandelbrot fractal
--------------------------------------------

Separate exercise notebook or use your own Mandelbrot code

1. Profile the code, find the bottlenecks
2. Use Numba to speed up the code
3. Compare the timing

In [ ]:
@jit
def add(a, b):
    return a + b


#def prod(a,b):
#    return a*b

In [ ]:
%%timeit -r 1
add(1, 1)

535 ns ± 0 ns per loop (mean ± std. dev. of 1 run, 1000000 loops each)


Numba examines Python bytecode and then translates this into an 'intermediate representation'.  To view this IR, run (compile) `add` and you can access the `inspect_types` method.

In [ ]:
add.inspect_types()

add (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-10-485ff261912d>
# --- LINE 1 --- 

@jit

# --- LINE 2 --- 

def add(a, b):

    # --- LINE 3 --- 
    # label 0
    #   a = arg(0, name=a)  :: int64
    #   b = arg(1, name=b)  :: int64
    #   $6binary_add.2 = a + b  :: int64
    #   del b
    #   del a
    #   $8return_value.3 = cast(value=$6binary_add.2)  :: int64
    #   del $6binary_add.2
    #   return $8return_value.3

    return a + b




In [ ]:
import pprint
pprint.pprint(add.inspect_asm())

{(int64, int64): '\t.text\n'
                 '\t.file\t"<string>"\n'
                 '\t.globl\t'
                 '_ZN8__main__3addB2v2B44c8tJTIcFHzwl2ILiXkcBLaFgF0XXBPQ5mKUJAA_3d_3dExx\n'
                 '\t.p2align\t4, 0x90\n'
                 '\t.type\t'
                 '_ZN8__main__3addB2v2B44c8tJTIcFHzwl2ILiXkcBLaFgF0XXBPQ5mKUJAA_3d_3dExx,@function\n'
                 '_ZN8__main__3addB2v2B44c8tJTIcFHzwl2ILiXkcBLaFgF0XXBPQ5mKUJAA_3d_3dExx:\n'
                 '\taddq\t%rcx, %rdx\n'
                 '\tmovq\t%rdx, (%rdi)\n'
                 '\txorl\t%eax, %eax\n'
                 '\tretq\n'
                 '.Lfunc_end0:\n'
                 '\t.size\t'
                 '_ZN8__main__3addB2v2B44c8tJTIcFHzwl2ILiXkcBLaFgF0XXBPQ5mKUJAA_3d_3dExx, '
                 '.Lfunc_end0-_ZN8__main__3addB2v2B44c8tJTIcFHzwl2ILiXkcBLaFgF0XXBPQ5mKUJAA_3d_3dExx\n'
                 '\n'
                 '\t.globl\t'
                 '_ZN7cpython8__main__3addB2v2B44c8tJTIcFHzwl2ILiXkcBLaFgF0XXBPQ5m

In [ ]:
add(1., 1.)

2.0

In [ ]:
add.inspect_types()

add (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-10-485ff261912d>
# --- LINE 1 --- 

@jit

# --- LINE 2 --- 

def add(a, b):

    # --- LINE 3 --- 
    # label 0
    #   a = arg(0, name=a)  :: int64
    #   b = arg(1, name=b)  :: int64
    #   $6binary_add.2 = a + b  :: int64
    #   del b
    #   del a
    #   $8return_value.3 = cast(value=$6binary_add.2)  :: int64
    #   del $6binary_add.2
    #   return $8return_value.3

    return a + b


add (float64, float64)
--------------------------------------------------------------------------------
# File: <ipython-input-10-485ff261912d>
# --- LINE 1 --- 

@jit

# --- LINE 2 --- 

def add(a, b):

    # --- LINE 3 --- 
    # label 0
    #   a = arg(0, name=a)  :: float64
    #   b = arg(1, name=b)  :: float64
    #   $6binary_add.2 = a + b  :: float64
    #   del b
    #   del a
    #   $8return_value.3 = cast(value=$6binary_add.2)  :: float64
    #   del $6binary_ad

In [ ]:
add("Hello ", "World!")

'Hello World!'

In [ ]:
add.inspect_types()

add (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-10-485ff261912d>
# --- LINE 1 --- 

@jit

# --- LINE 2 --- 

def add(a, b):

    # --- LINE 3 --- 
    # label 0
    #   a = arg(0, name=a)  :: int64
    #   b = arg(1, name=b)  :: int64
    #   $6binary_add.2 = a + b  :: int64
    #   del b
    #   del a
    #   $8return_value.3 = cast(value=$6binary_add.2)  :: int64
    #   del $6binary_add.2
    #   return $8return_value.3

    return a + b


add (float64, float64)
--------------------------------------------------------------------------------
# File: <ipython-input-10-485ff261912d>
# --- LINE 1 --- 

@jit

# --- LINE 2 --- 

def add(a, b):

    # --- LINE 3 --- 
    # label 0
    #   a = arg(0, name=a)  :: float64
    #   b = arg(1, name=b)  :: float64
    #   $6binary_add.2 = a + b  :: float64
    #   del b
    #   del a
    #   $8return_value.3 = cast(value=$6binary_add.2)  :: float64
    #   del $6binary_ad

Ok.  Numba is has correctly inferred the type of the arguments, defining things as `int64` and running smoothly.  

(What happens if you do `add(1., 1.)` and then `inspect_types`?)

In [ ]:
add(1.,1.)

2.0

In [ ]:
import pprint

In [ ]:
pprint.pprint(add.inspect_asm())

{(unicode_type, unicode_type): '\t.text\n'
                               '\t.file\t"<string>"\n'
                               '\t.globl\t'
                               '_ZN8__main__3addB2v4B44c8tJTIcFHzwl2ILiXkcBLaFgF0XXBPQ5mKUJAA_3d_3dE12unicode_type12unicode_type\n'
                               '\t.p2align\t4, 0x90\n'
                               '\t.type\t'
                               '_ZN8__main__3addB2v4B44c8tJTIcFHzwl2ILiXkcBLaFgF0XXBPQ5mKUJAA_3d_3dE12unicode_type12unicode_type,@function\n'
                               '_ZN8__main__3addB2v4B44c8tJTIcFHzwl2ILiXkcBLaFgF0XXBPQ5mKUJAA_3d_3dE12unicode_type12unicode_type:\n'
                               '\t.cfi_startproc\n'
                               '\tpushq\t%rbp\n'
                               '\t.cfi_def_cfa_offset 16\n'
                               '\t.cfi_offset %rbp, -16\n'
                               '\tmovq\t%rsp, %rbp\n'
                               '\t.cfi_def_cfa_register %rbp\n'
               

### What about the actual LLVM code?

You can see the actual LLVM code generated by Numba using the `inspect_llvm()` method.  Since it's a `dict`, doing the following will be slightly more visually friendly.

In [ ]:
for k, v in add.inspect_llvm().items():
    print(k, v)

(int64, int64) ; ModuleID = 'add'
source_filename = "<string>"
target datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

@_ZN08NumbaEnv8__main__3addB2v2B44c8tJTIcFHzwl2ILiXkcBLaFgF0XXBPQ5mKUJAA_3d_3dExx = common local_unnamed_addr global i8* null
@.const.add = internal constant [4 x i8] c"add\00"
@PyExc_RuntimeError = external global i8
@".const.missing Environment: _ZN08NumbaEnv8__main__3addB2v2B44c8tJTIcFHzwl2ILiXkcBLaFgF0XXBPQ5mKUJAA_3d_3dExx" = internal constant [102 x i8] c"missing Environment: _ZN08NumbaEnv8__main__3addB2v2B44c8tJTIcFHzwl2ILiXkcBLaFgF0XXBPQ5mKUJAA_3d_3dExx\00"

; Function Attrs: nofree norecurse nounwind writeonly
define i32 @_ZN8__main__3addB2v2B44c8tJTIcFHzwl2ILiXkcBLaFgF0XXBPQ5mKUJAA_3d_3dExx(i64* noalias nocapture %retptr, { i8*, i32, i8* }** noalias nocapture readnone %excinfo, i64 %arg.a, i64 %arg.b) local_unnamed_addr #0 {
entry:
  %.6 = add nsw i64 %arg.b, %arg.a
  store i64 %.

## But there's a caveat
Now, watch what happens when we try to do something that is natural in Python, but not particularly mathematically sound:

In [ ]:
from numba import njit

In [ ]:
def add_strings(a, b):
    return a + b

In [ ]:
add_strings_jit = njit()(add_strings)

In [ ]:
add_strings_jit('a', 'b')

'ab'

In [ ]:
@njit
def create_dict():
    a = [0, 1, (10,20), {"a":10}]
    return a

In [ ]:
create_dict()[0]

<ipython-input-40-ee656fbaf83e>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function create_dict failed at nopython mode lowering due to: unhashable type: 'list'
  @jit
/usr/local/lib/python3.10/dist-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "create_dict" was compiled in object mode without forceobj=True.

File "<ipython-input-40-ee656fbaf83e>", line 2:
@jit
def create_dict():
^

  warnings.warn(errors.NumbaWarning(warn_msg,
/usr/local/lib/python3.10/dist-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "<ipython-input-40-ee656fbaf83e>", line 2:
@jit
def create_dict():
^

  warnings.warn(errors.Numb

0

In [ ]:
add_strings_jit.inspect_types()

add_strings (unicode_type, unicode_type)
--------------------------------------------------------------------------------
# File: <ipython-input-24-d1008a9d4aa2>
# --- LINE 1 --- 
# label 0

def add_strings(a, b):

    # --- LINE 2 --- 
    #   a = arg(0, name=a)  :: unicode_type
    #   b = arg(1, name=b)  :: unicode_type
    #   $0.3 = a + b  :: unicode_type
    #   del b
    #   del a
    #   $0.4 = cast(value=$0.3)  :: unicode_type
    #   del $0.3
    #   return $0.4

    return a + b




In [ ]:
@jit(int64, int64)

In [ ]:
@njit(cache=True)
def minus(a, b):
    return a-b

In [ ]:
minus(10., 20.)

-10.0

### Compiler options

```python
cache=True
```

if you don't want to always want to get dinged by the compilation time for every run. This will actually save the compiled function into something like a `pyc` file in your `__pycache__` directory, so even between sessions you should have nice fast performance.

In [ ]:
!ls -la

total 16
drwxr-xr-x 1 root root 4096 Nov 14 14:32 .
drwxr-xr-x 1 root root 4096 Nov 16 14:12 ..
drwxr-xr-x 4 root root 4096 Nov 14 14:31 .config
drwxr-xr-x 1 root root 4096 Nov 14 14:32 sample_data


## N-Body problems

Many physical problems require the evaluation of all pairwise interactions of a large number of particles, so-called N-body problems. These problems arise in molecular dynamics, astrodynamics and electromagnetics among others.

Their pairwise interactions can be expressed as:

\begin{equation}
f_i = \sum_{j=1}^n{P \left(\boldsymbol{x}_i, \boldsymbol{x}_j \right)w_j} \ \ \ \text{for } i=1,2,...,n
\end{equation}

*  where subscripts $i$,  $j$ respectively denote *target* and *source*
*  $f_i$ can be a *potential* (or *force*) at target point $i$
*  $w_j$ is the *source weight*
*  $\boldsymbol{x}_i, \boldsymbol{x}_j$ are the *spatial positions* of particles
*  $P \left(\boldsymbol{x}_i, \boldsymbol{x}_j \right)$ is the *interaction kernel*.

In order to evalute the potential $f_i$ at a target point $i$, we have to loop over each source particle $j$. Since there are $n$ target points $i$, this 'brute-force' approach costs $\mathcal{O} \left(n^2 \right)$ operations.

One possible approach in this kind of problem is to define a few classes, say `Point` and `Particle` and then loop over the objects and perform the necessary point-to-point calculations.

In [ ]:
class Point():
    """
    Arguments:
        domain: the domain of random generated coordinates x,y,z,
                default=1.0

    Attributes:
        x, y, z: coordinates of the point
    """
    def __init__(self, domain=1.0):
        self.x = domain * numpy.random.random()
        self.y = domain * numpy.random.random()
        self.z = domain * numpy.random.random()

    def distance(self, other):
        return ((self.x - other.x)**2 +
                (self.y - other.y)**2 +
                (self.z - other.z)**2)**.5

In [ ]:
Barnes-Hut

In [ ]:
class Particle(Point):
    """
    Attributes:
        m: mass of the particle
        phi: the potential of the particle
    """

    def __init__(self, domain=1.0, m=1.0):
        Point.__init__(self, domain)
        self.m = m
        self.phi = 0.

Now we create a list of `n` random particles, define a function to calculate their interaction via direct summation and run!

In [ ]:
n = 1000
particles = [Particle(m = 1 / n) for i in range(n)]

In [ ]:
particles[10].m

0.001

In [ ]:
def direct_sum(particles):
    """
    Calculate the potential at each particle
    using direct summation method.

    Arguments:
        particles: the list of particles

    """
    for i, target in enumerate(particles):
        for source in (particles[:i] + particles[i+1:]):
            r = target.distance(source)
            target.phi += source.m / r

In [ ]:
direct_sum(particles)

In [ ]:
orig_time = %timeit -o direct_sum(particles)

602 ms ± 9.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f direct_sum direct_sum(particles)

## How do we use Numba on this problem?
Problem: Numba doesn't support jitting native Python classes.  There is a `jit_class` structure in Numba but it's still in early development.

But it's nice to have attributes for literate programming.

Solution: NumPy custom dtypes.

In [ ]:
particle_dtype = numpy.dtype({'names':['x','y','z','m','phi'],
                             'formats':[numpy.double,
                                        numpy.double,
                                        numpy.double,
                                        numpy.double,
                                        numpy.double]})

In [ ]:
myarray = numpy.ones(3, dtype=particle_dtype)

In [ ]:
myarray[0]["z"]

1.0

In [ ]:
myarray['x']=numpy.ones(3)*10

In [ ]:
myarray

array([(10., 1., 1., 1., 1.), (10., 1., 1., 1., 1.),
       (10., 1., 1., 1., 1.)],
      dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('m', '<f8'), ('phi', '<f8')])

### Exercise 1

Write a function `create_n_random_particles` that takes the arguments `n` (number of particles), `m` (mass of every particle) and a domain within to generate a random number (as in the class above).
It should create an array with `n` elements and `dtype=particle_dtype` and then return that array.

For each particle, the mass should be initialized to the value of `m` and the potential `phi` initialized to zero.

For the `x` component of a given particle `p`, you might do something like

```python
p['x'] = domain * numpy.random.random()
```

In [ ]:
@njit
def create_n_random_particles(n, m, domain=1):
    '''
    Creates `n` particles with mass `m` with random coordinates
    between 0 and `domain`
    '''
    parts = numpy.zeros((n), dtype=particle_dtype)


    parts['x'] = numpy.random.random(m)*domain
    ### your code
    ### your code
    ### your code

    return parts   #parts is an array of particles

In [ ]:
# test it
parts = create_n_random_particles(1000, .001, 1)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mInvalid use of Function(np.random.random) with argument(s) of type(s): (float64)
Known signatures:
 * () -> float64
 * parameterized
[1mIn definition 0:[0m
[1m    TypeError: object of type 'Float' has no len()[0m
    raised from /Users/sergey/opt/anaconda3/lib/python3.7/site-packages/numba/typing/randomdecl.py:38
[1mIn definition 1:[0m
[1m    TypeError: object of type 'Float' has no len()[0m
    raised from /Users/sergey/opt/anaconda3/lib/python3.7/site-packages/numba/typing/randomdecl.py:38
[1mThis error is usually caused by passing an argument of a type that is unsupported by the named function.[0m[0m
[0m[1m[1] During: resolving callee type: Function(np.random.random)[0m
[0m[1m[2] During: typing of call at <ipython-input-73-2e691807810d> (10)
[0m
[1m
File "<ipython-input-73-2e691807810d>", line 10:[0m
[1mdef create_n_random_particles(n, m, domain=1):
    <source elided>

[1m    pa

### Exercise 2

Write a JITted function `distance` to calculate the distance between two particles of dtype `particle_dtype`

Here's the `distance` method from the `Particle` class as a reference:

```python
def distance(self, other):
        return ((self.x - other.x)**2 +
                (self.y - other.y)**2 +
                (self.z - other.z)**2)**.5
```

In [ ]:
def distance(part1, part2):
    '''calculate the distance between two particles'''

    # your code here

In [ ]:
# test it

distance(parts[0], parts[1])

### Exercise 3
Modify the original `direct_sum` function (copied below for reference) to instead work a NumPy array of particles.  Loop over each element in the array and calculate its total potential.

```python
def direct_sum(particles):
    """
    Calculate the potential at each particle
    using direct summation method.

    Arguments:
        particles: the list of particles

    """
    for i, target in enumerate(particles):
        for source in (particles[:i] + particles[i+1:]):
            r = target.distance(source)
            target.phi += source.m / r

In [ ]:
def direct_sum(particles):
    # take it away